# 🎮 Pokémon Battle Arena using Reinforcement Learning
This project implements a PPO-based RL agent that learns to battle Pokémon opponents using a type-based combat system.
Features:
- Reinforcement Learning (PPO, Stable-Baselines3)
- Type advantage logic (Fire, Water, Grass, Electric)
- Cinematic inline MP4 battle visualization
- Dynamic opponents and randomized moves


In [62]:
!pip install -q gymnasium stable-baselines3[extra] moviepy pillow opencv-python requests torch


In [63]:
# Imports
import os, random, warnings, cv2, numpy as np, requests
from datetime import datetime
from moviepy.editor import VideoFileClip, concatenate_videoclips, ImageClip
from IPython.display import Video, display
from PIL import Image as PILImage, ImageDraw
from io import BytesIO
from stable_baselines3 import PPO
from gymnasium import spaces
import gymnasium as gym

warnings.filterwarnings("ignore")
os.makedirs("battle_videos", exist_ok=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cuda


In [64]:
from IPython.display import HTML

html = """
<h3 style='color:#FFD700;'>📘 Pokédex — Choose Your Pokémon (1–151)</h3>
<div style='width:820px; height:420px; overflow-y:scroll; border:2px solid #FFD700; padding:10px; background-color:#111;'>
<table style='border-collapse:collapse;'><tr>
"""
for pid in range(1, 152):
    sprite_url = f"https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/{pid}.png"
    html += f"<td style='text-align:center; padding:6px;'><img src='{sprite_url}' width='64'><br><b style='color:#FFF'>{pid}</b></td>"
    if pid % 10 == 0:
        html += "</tr><tr>"
html += "</tr></table></div>"
display(HTML(html))

1,2,3,4,5,6,7,8,9,10
11,12,13,14,15,16,17,18,19,20
21,22,23,24,25,26,27,28,29,30
31,32,33,34,35,36,37,38,39,40
41,42,43,44,45,46,47,48,49,50
51,52,53,54,55,56,57,58,59,60
61,62,63,64,65,66,67,68,69,70
71,72,73,74,75,76,77,78,79,80
81,82,83,84,85,86,87,88,89,90
91,92,93,94,95,96,97,98,99,100
101,102,103,104,105,106,107,108,109,110


In [65]:
try:
    USER_POKEMON_ID = int(input("Enter your Pokémon ID (1–151): ") or 25)
    if not (1 <= USER_POKEMON_ID <= 151):
        raise ValueError
except ValueError:
    USER_POKEMON_ID = 25

print(f"⚡ You chose Pokémon ID: {USER_POKEMON_ID}")


Enter your Pokémon ID (1–151): 6
⚡ You chose Pokémon ID: 6


In [66]:
SPRITE_BASE = "https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/"

TYPE_MAP = {
    "fire": ["Flamethrower 🔥", "Fire Blast 🔥"],
    "water": ["Hydro Pump 💧", "Aqua Jet 💧"],
    "grass": ["Vine Whip 🌿", "Leaf Storm 🍃"],
    "electric": ["Thunderbolt ⚡", "Spark ⚡"],
    "normal": ["Tackle 💥", "Slash ⚔️"]
}

POKEMON_TYPES = {
    1: "grass", 4: "fire", 6: "fire", 7: "water", 9: "water", 12: "grass", 13: "grass",
    16: "normal", 17: "normal", 18: "normal", 19: "normal", 20: "normal", 23: "normal",
    24: "normal", 25: "electric", 26: "electric", 37: "fire", 38: "fire", 39: "normal",
    40: "normal", 52: "normal", 54: "water", 58: "fire", 59: "fire", 63: "normal",
    66: "normal", 72: "water", 74: "normal", 77: "fire", 79: "water", 81: "electric",
    90: "water", 92: "normal", 94: "normal", 95: "normal", 100: "electric",
    120: "water", 129: "water", 133: "normal", 134: "water", 135: "electric",
    136: "fire", 143: "normal", 147: "water"
}

TYPE_ADV = {
    "fire": {"grass": 1.5, "water": 0.5},
    "water": {"fire": 1.5, "grass": 0.5},
    "grass": {"water": 1.5, "fire": 0.5},
    "electric": {"water": 1.5, "grass": 1.0},
}

def get_type(pid):
    return POKEMON_TYPES.get(pid, "normal")

def type_multiplier(attacker, defender):
    a_type, d_type = get_type(attacker), get_type(defender)
    return TYPE_ADV.get(a_type, {}).get(d_type, 1.0)

def random_move(pid):
    return random.choice(TYPE_MAP.get(get_type(pid), ["Tackle 💥"]))


In [67]:
class PokemonBattleEnv(gym.Env):
    def __init__(self, agent_poke_id=25, opp_poke_id=4):
        super().__init__()
        self.agent_id, self.opp_id = agent_poke_id, opp_poke_id
        self.agent_sprite, self.opp_sprite = self.fetch_sprite(agent_poke_id), self.fetch_sprite(opp_poke_id)
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Box(low=0, high=100, shape=(4,), dtype=np.float32)
        self.reset()

    def fetch_sprite(self, pid):
        try:
            img = PILImage.open(BytesIO(requests.get(f"{SPRITE_BASE}{pid}.png", timeout=5).content)).convert("RGBA")
            return img
        except:
            return PILImage.new("RGBA", (96,96), (0,0,0,0))

    def reset(self, *, seed=None, options=None):
        self.agent_hp, self.agent_energy = 100., 100.
        self.opp_hp, self.opp_energy = 100., 100.
        self.turn, self.total_damage, self.total_taken = 0, 0, 0
        return self._get_obs(), {}

    def _get_obs(self):
        return np.array([self.agent_hp, self.agent_energy, self.opp_hp, self.turn], dtype=np.float32)

    def step(self, action):
        self.turn += 1
        reward, done = 0, False
        a_mult = type_multiplier(self.agent_id, self.opp_id)
        o_mult = type_multiplier(self.opp_id, self.agent_id)

        # Agent move
        if action == 0:
            dmg = random.randint(10,20)
            self.opp_hp -= dmg * a_mult; self.agent_energy -= 10; reward += 0.5*dmg
            move_text = f"{random_move(self.agent_id)} → {int(dmg*a_mult)} dmg!"
            self.total_damage += dmg * a_mult
        elif action == 1:
            dmg = random.randint(25,40)
            self.opp_hp -= dmg * a_mult; self.agent_energy -= 25; reward += 0.8*dmg
            move_text = f"{random_move(self.agent_id)} → {int(dmg*a_mult)} dmg!"
            self.total_damage += dmg * a_mult
        elif action == 2:
            move_text = "Guard 🛡️"
        elif action == 3:
            heal = random.randint(10,25)
            self.agent_hp = min(100,self.agent_hp+heal)
            move_text = f"Recover 💖 +{heal} HP"
        else:
            move_text = "Wait ⏳"

        # Opponent move
        if self.opp_hp > 0:
            if self.opp_hp < 30 and random.random() < 0.5:
                heal = random.randint(15,25)
                self.opp_hp = min(100, self.opp_hp + heal)
            else:
                dmg = random.randint(10,30)
                dmg *= o_mult
                if action == 2: dmg /= 2
                self.agent_hp -= dmg
                self.total_taken += dmg
        reward -= 0.5
        done = self.agent_hp <= 0 or self.opp_hp <= 0 or self.turn >= 50
        return self._get_obs(), reward, done, False, {"action_text": move_text}


In [68]:
from stable_baselines3.common.vec_env import DummyVecEnv

def make_env():
    def _init():
        env = PokemonBattleEnv(USER_POKEMON_ID, random.choice(range(1,152)))
        return env
    return _init

env = DummyVecEnv([make_env()])
model = PPO("MlpPolicy", env, verbose=0, device=device)
print("🚀 Training Agent...")
model.learn(total_timesteps=100_000)
print("✅ Training complete!")


🚀 Training Agent...
✅ Training complete!


In [69]:
from IPython.display import Video, display

# Randomized opponents by type diversity
all_ids = list(POKEMON_TYPES.keys())
opponents = random.sample(all_ids, 5)
results = []

for opp_id in opponents:
    opp_name = f"Pokémon {opp_id}"
    print(f"\n⚔️ Battle: Pokémon {USER_POKEMON_ID} vs {opp_name} ⚔️\n{'='*60}")

    env = PokemonBattleEnv(agent_poke_id=USER_POKEMON_ID, opp_poke_id=opp_id)
    obs, _ = env.reset()
    frames, log = [], []

    for t in range(60):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _, info = env.step(int(action))
        text = info.get("action_text", f"Turn {t+1}")
        frame = make_battle_frame(env.agent_sprite, env.opp_sprite,
                                  int(env.agent_hp), int(env.agent_energy),
                                  int(env.opp_hp), env.turn, text)
        frames.append(np.array(frame))
        log.append(f"Turn {t+1}: {text}")
        if done:
            break

    for l in log: print(l)

    winner = "Agent (You)" if env.opp_hp <= 0 else "Opponent"
    print(f"\n🏆 Winner: {winner}\n{'-'*60}")
    print(f"💥 Damage Dealt: {int(env.total_damage)} | 💔 Damage Taken: {int(env.total_taken)} | Turns: {env.turn}\n")

    base_path = f"battle_videos/battle_vs_{opp_name}.mp4"
    h, w, _ = frames[0].shape
    writer = cv2.VideoWriter(base_path, cv2.VideoWriter_fourcc(*'mp4v'), 6, (w,h))
    for f in frames:
        writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
    writer.release()

    intro_clip = ImageClip(np.array(make_battle_frame(env.agent_sprite, env.opp_sprite,100,100,100,0,f"⚔️ Pokémon Battle ⚔️\\n{USER_POKEMON_ID} vs {opp_name}"))).set_duration(2)
    battle_clip = VideoFileClip(base_path)
    outro_clip = ImageClip(np.array(make_battle_frame(env.agent_sprite, env.opp_sprite,int(env.agent_hp),int(env.agent_energy),int(env.opp_hp),env.turn,f"🏆 Winner: {winner}"))).set_duration(2)

    final_clip = concatenate_videoclips([intro_clip, battle_clip, outro_clip])
    final_path = f"battle_videos/final_battle_vs_{opp_name}_{datetime.now().strftime('%H%M%S')}.mp4"
    final_clip.write_videofile(final_path, codec='libx264', audio=False, verbose=False, logger=None)

    print(f"🎬 Saved polished video → {final_path}")
    display(Video(final_path, embed=True))
    results.append((opp_name, winner, int(env.total_damage), int(env.total_taken)))

print("\n🏁 Tournament Summary:")
for name, win, dmg, taken in results:
    print(f"{name:12} -> Winner: {win} | 💥 Damage: {dmg} | 💔 Taken: {taken}")



⚔️ Battle: Pokémon 6 vs Pokémon 26 ⚔️
Turn 1: Flamethrower 🔥 → 27 dmg!
Turn 2: Fire Blast 🔥 → 33 dmg!
Turn 3: Fire Blast 🔥 → 38 dmg!
Turn 4: Recover 💖 +13 HP
Turn 5: Recover 💖 +19 HP
Turn 6: Recover 💖 +12 HP
Turn 7: Recover 💖 +25 HP
Turn 8: Flamethrower 🔥 → 27 dmg!
Turn 9: Recover 💖 +23 HP
Turn 10: Recover 💖 +23 HP
Turn 11: Recover 💖 +12 HP
Turn 12: Recover 💖 +17 HP
Turn 13: Fire Blast 🔥 → 33 dmg!

🏆 Winner: Agent (You)
------------------------------------------------------------
💥 Damage Dealt: 158 | 💔 Damage Taken: 191 | Turns: 13

🎬 Saved polished video → battle_videos/final_battle_vs_Pokémon 26_092117.mp4



⚔️ Battle: Pokémon 6 vs Pokémon 66 ⚔️
Turn 1: Flamethrower 🔥 → 26 dmg!
Turn 2: Flamethrower 🔥 → 36 dmg!
Turn 3: Flamethrower 🔥 → 33 dmg!
Turn 4: Recover 💖 +21 HP
Turn 5: Recover 💖 +25 HP
Turn 6: Flamethrower 🔥 → 33 dmg!
Turn 7: Recover 💖 +17 HP
Turn 8: Fire Blast 🔥 → 25 dmg!
Turn 9: Flamethrower 🔥 → 30 dmg!
Turn 10: Recover 💖 +17 HP
Turn 11: Recover 💖 +25 HP
Turn 12: Fire Blast 🔥 → 27 dmg!
Turn 13: Recover 💖 +17 HP
Turn 14: Recover 💖 +15 HP
Turn 15: Flamethrower 🔥 → 33 dmg!
Turn 16: Recover 💖 +11 HP
Turn 17: Recover 💖 +11 HP
Turn 18: Recover 💖 +10 HP
Turn 19: Recover 💖 +21 HP
Turn 20: Flamethrower 🔥 → 35 dmg!
Turn 21: Recover 💖 +22 HP
Turn 22: Recover 💖 +22 HP
Turn 23: Recover 💖 +18 HP
Turn 24: Recover 💖 +12 HP
Turn 25: Fire Blast 🔥 → 28 dmg!
Turn 26: Recover 💖 +24 HP
Turn 27: Recover 💖 +24 HP
Turn 28: Fire Blast 🔥 → 40 dmg!

🏆 Winner: Agent (You)
------------------------------------------------------------
💥 Damage Dealt: 346 | 💔 Damage Taken: 287 | Turns: 28

🎬 Saved polished video 


⚔️ Battle: Pokémon 6 vs Pokémon 12 ⚔️
Turn 1: Flamethrower 🔥 → 42 dmg!
Turn 2: Flamethrower 🔥 → 48 dmg!
Turn 3: Recover 💖 +16 HP
Turn 4: Recover 💖 +18 HP
Turn 5: Recover 💖 +12 HP
Turn 6: Recover 💖 +13 HP
Turn 7: Recover 💖 +13 HP
Turn 8: Recover 💖 +14 HP
Turn 9: Recover 💖 +11 HP
Turn 10: Recover 💖 +20 HP
Turn 11: Recover 💖 +25 HP
Turn 12: Recover 💖 +15 HP
Turn 13: Recover 💖 +20 HP
Turn 14: Recover 💖 +25 HP
Turn 15: Recover 💖 +18 HP
Turn 16: Recover 💖 +12 HP
Turn 17: Recover 💖 +22 HP
Turn 18: Recover 💖 +20 HP
Turn 19: Recover 💖 +10 HP
Turn 20: Recover 💖 +11 HP
Turn 21: Recover 💖 +22 HP
Turn 22: Recover 💖 +18 HP
Turn 23: Recover 💖 +21 HP
Turn 24: Recover 💖 +10 HP
Turn 25: Recover 💖 +12 HP
Turn 26: Recover 💖 +12 HP
Turn 27: Recover 💖 +23 HP
Turn 28: Recover 💖 +17 HP
Turn 29: Recover 💖 +10 HP
Turn 30: Recover 💖 +23 HP
Turn 31: Recover 💖 +24 HP
Turn 32: Recover 💖 +25 HP
Turn 33: Recover 💖 +21 HP
Turn 34: Recover 💖 +25 HP
Turn 35: Recover 💖 +13 HP
Turn 36: Recover 💖 +13 HP
Turn 37: Recover 💖


⚔️ Battle: Pokémon 6 vs Pokémon 9 ⚔️
Turn 1: Flamethrower 🔥 → 17 dmg!
Turn 2: Flamethrower 🔥 → 16 dmg!
Turn 3: Flamethrower 🔥 → 13 dmg!
Turn 4: Fire Blast 🔥 → 20 dmg!

🏆 Winner: Opponent
------------------------------------------------------------
💥 Damage Dealt: 66 | 💔 Damage Taken: 100 | Turns: 4

🎬 Saved polished video → battle_videos/final_battle_vs_Pokémon 9_092119.mp4



⚔️ Battle: Pokémon 6 vs Pokémon 37 ⚔️
Turn 1: Fire Blast 🔥 → 29 dmg!
Turn 2: Flamethrower 🔥 → 29 dmg!
Turn 3: Fire Blast 🔥 → 32 dmg!
Turn 4: Fire Blast 🔥 → 27 dmg!
Turn 5: Recover 💖 +19 HP
Turn 6: Flamethrower 🔥 → 35 dmg!

🏆 Winner: Agent (You)
------------------------------------------------------------
💥 Damage Dealt: 152 | 💔 Damage Taken: 57 | Turns: 6

🎬 Saved polished video → battle_videos/final_battle_vs_Pokémon 37_092119.mp4



🏁 Tournament Summary:
Pokémon 26   -> Winner: Agent (You) | 💥 Damage: 158 | 💔 Taken: 191
Pokémon 66   -> Winner: Agent (You) | 💥 Damage: 346 | 💔 Taken: 287
Pokémon 12   -> Winner: Opponent | 💥 Damage: 90 | 💔 Taken: 499
Pokémon 9    -> Winner: Opponent | 💥 Damage: 66 | 💔 Taken: 100
Pokémon 37   -> Winner: Agent (You) | 💥 Damage: 152 | 💔 Taken: 57
